In [2]:
# from https://gamedev.stackexchange.com/questions/586/what-is-the-fastest-way-to-work-out-2d-bounding-box-intersection
def bounding_box_in_window(window, box):
    return not (box[0] > window[2]
        or box[2] < window[0]
        or box[1] > window[3]
        or box[3] < window[1])

In [16]:
import os
from PIL import Image
import PIL.ImageDraw as ImageDraw
import numpy as np
import time

annotation_path = 'train.txt'
with open(annotation_path) as f:
    lines = f.readlines()

max_boxes = 20
floydhub_data_path = "/data/map-images/yolo3-sliding-windows/"
      
anots_file_contents = ""    

for line in lines:
    line = line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = (416, 416)
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])
    
    windows = []
    boxes_per_window = []
    current_window = 0
    for y in range(0, ih, h):
        for x in range(0, iw, w):
            area = (x, y, x+w, y+h)
            window = image.crop(area)
            image_data = np.array(window)/255.
            windows.append(image_data)
            
            # save new image
            new_path = "/home/sean/Documents/indystudy/data/map-images/yolo3-sliding-windows/"
            image_name = line[0].split('/')[-1].split('.')[0]
            image_ext = line[0].split('/')[-1].split('.')[1]
            new_name =  image_name + "_" + str(current_window) + '.' + image_ext
            
            #window.save(os.path.join(new_path, new_name))
            
            anot_line = os.path.join(new_path, new_name)

            # get boxes for window
            boxes_for_window = np.zeros((max_boxes,5))
            if len(box)>0:
                boxes_added = 0
                for bb in box:
                    if boxes_added < max_boxes and bounding_box_in_window(area, bb[:4]):
                        
                        # adjust bounding box coords to window
                        adjusted_bb = [bb[0]-x, bb[1]-y, bb[2]-x, bb[3]-y, 0]
                        
                        # add box to list for this window
                        boxes_for_window[boxes_added] = adjusted_bb
                        boxes_added += 1
                        
                        anot_line += " " + ",".join(list(map(str, adjusted_bb)))
            
            if boxes_added > 0:
                anots_file_contents += anot_line + "\n"
            
            boxes_per_window.append(boxes_for_window)
            
            current_window += 1
            
with open("window_train.txt", "w") as text_file:
    text_file.write(anots_file_contents)

In [11]:
for line in lines:
    line = line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = (416, 416)
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])
    
    draw = ImageDraw.Draw(image)
    
    for bb in box:
        x_min, y_min, x_max, y_max, _ = bb
        draw.line([
            (x_min, y_min),
            (x_min, y_max),
            (x_max, y_max),
            (x_max, y_min),
            (x_min, y_min)
        ], width = 5, fill="red")
            
    image.show()
    time.sleep(100)

KeyboardInterrupt: 